In [ ]:
import torch
import math
import matplotlib.pyplot as plt

## 关于位置编码的可视化

In [ ]:
def get_positional_encoding(seq_len, d_model):
    pe = torch.zeros(seq_len, d_model)
    position = torch.arange(0, seq_len, dtype=torch.float).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    return pe

seq_len = 1000    
d_model = 50000   

# 生成位置编码
pe = get_positional_encoding(seq_len, d_model)

# 可视化
plt.figure(figsize=(10, 6))
plt.imshow(pe, aspect='auto', cmap='coolwarm')
plt.colorbar(label="Value")
plt.xlabel("Embedding Dimension (d_model index)")
plt.ylabel("Sequence Position (token index)")
plt.title("Positional Encoding Heatmap")
plt.show()


## 关于广播机制

对于 tensor 计算 , 比如形状为 ( 5 , 6 , 7 ) 的 tensor , 其第 0 , 1 , 2 维是  5 , 6 , 7 ( 也就是顺序的 ) , 而计算时则是先从最后的维度反向向前 .

( 刚开始容易与只有二维的矩阵混淆 , 或者向量中表示一个样本点的到底是一个列还是一个行 . 可以理解为代码里的层级关系为比如 : 5 个块 , 一个块里有 6 **行** , 一行里有 7 个元素 )

( 而一般来说第 0 个维度表示 batch_size 大小 , 而后面在 nlp 里经常是 seq_len , features , 因为这个更容易与 batch_size 一起表现出层层递减的关系 , 也算一个经验习惯 , 但真的反过来没人管 , 自己的计算过程能前后没有歧义才重要 )

而直观上广播就是补足 , 比如 $Wx + b$ 中 $b$ 是一个向量 , 但做加法时会广播其为一个与 $W$ 同尺寸的张量 , 并将每一个 多出来的一维度用那一行的 " 单位量 " 填满 .

如此 , torch ( 以及 numpy ) 规定的能被识别的合法广播含 : 
1. tensor 至少一个维度 ( 显然 ) 
2. 从尾部维度开始 , 维度满足 : 或相等 , 或其中一个为 1 , 或其中一个不存在

In [ ]:
# 合法 : 完全相等
x=torch.empty(5,7,3)
y=torch.empty(5,7,3)

# 合法 : 其中一个不存在
x=torch.empty(5,3,4,1)
y=torch.empty(  3,1,1)

# 不合法 : x 维度不匹配
# (注意 0 代表的是一个 0 的维度而不是维度为空) 
x=torch.empty((0,))
y=torch.empty(2,2)

# 合法 : 维度为一以及维度不存在
x=torch.empty((1,))
y=torch.empty(2,2)

# 不合法 : 维度不相等
x=torch.empty(5,2,4,1)
y=torch.empty(  3,1,1)

广播计算规则 : 
1. 唯独数量不等时 , 在尾部补 1 使得维度相等 
2. 每个维度 ( 其实是维度 1 的地方 ) , 维度取最大值 ( 另一方 )

In [ ]:
x=torch.empty(5,1,4,1)
y=torch.empty(3,1,1)
(x+y).size()
# >>> torch.Size([5, 3, 4, 1])

# 反例
x=torch.empty(5,2,4,1)
y=torch.empty(3,1,1)
(x+y).size()
# >>> RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 1

## 关于 `.matmul()`

并不真的存在直接由矩阵乘法映射过去的张量乘法 , 所谓张量乘法会被 torch 处理为低纬度的矩阵/向量乘法 . 

具体规则如下 : 

( 当然可以直接用数学与图的行列表示 , 但在这里非常不直观还容易弄混 , 所以尝试用第 x 维由内向外表示维度 )

In [ ]:
# 向量 @ 向量 (n,) @ (n,) 
# 点乘 --> tensor封装的标量
x = torch.tensor([2, 4, 6, 8], dtype=torch.int)
y = torch.tensor([1, 2 ,7, 5], dtype=torch.int)
x @ y 
# >>> tensor(92, dtype=torch.int32)

# 向量 @ 矩阵 (n,) @ (n, m) ---> (m,)
# 将矩阵的第0维度的第i个数遍历地取出组成(n,)的矩阵,与向量相乘得到第i个位置的值
# 或者说将其前面加入维度1,进行矩阵乘法,结束后再删去那个维度
x = torch.tensor([2, 4, 6, 8], dtype=torch.int)
y = torch.tensor([[1, 2 ,7, 5], [1, 1, 1, 1]], dtype=torch.int).transpose(0, 1)
x @ y
# >>> tensor([92, 20], dtype=torch.int32)

# 矩阵 @ 向量 (m, n) @ (n,) ---> (m,)
# 最低维进行逐个点积,得到的当下位置所拥值,逐个排序
x = torch.tensor([2, 4, 6, 8], dtype=torch.int)
y = torch.tensor([[1, 2 ,7, 5], [1, 1, 1, 1]], dtype=torch.int)
y @ x
# >>> ttensor([92, 20], dtype=torch.int32) , 与上者等价

# 矩阵 @ 矩阵 (m, n) @ (n, r) ---> (m, r)
# 这时将左至右列为外至内来计算,用数学计算,再还原回去更明白些 
x = torch.tensor([[1,2],[3,4]], dtype=torch.int)
y = torch.tensor([[5,6],[7,8]], dtype=torch.int)
x @ y
# >>> tensor([[19, 22],
#             [43, 50]], dtype=torch.int32)

# 张量参与 : 取最后(最内)两个维度(如果有向量,则广播),进行标准运算.前面则理解为"批处理"

## 关于参数与子模块注册

它们都发生于继承了 nn.Module 模块成员被赋予成员属性时 , 也即比如 `self.weight = nn.Parameter(torch.randn(out_dim, in_dim))` , `self.layer1 = nn.Linear(10, 20)` 时 . 而这些被注册的量会被分别存到 self._modules 和 self._parameters 字典中 . 

作用是 , 一方面 , 模型保存时 ( 在state_dict() ) 能包含这些参数 / 模块 , 且优化器能获取所有可训练参数 , `.to(device) / .eval() / .train()` 能递归应用到所有子模块 

参数注册需要在声明时标注 `nn.Parameter()` , 或者单独在 `self.bias = torch.xxx` 后补上 `self.register_parameter('bias', bias)`

而子模块注册会在添加继承 `nn.Module` 的成员属性时自动注册 , 类似地 , 也能单独注册 `self.register_module(f'layer{i}', nn.Linear(10, 10))`
